# Group assignment 1.2: Numerical integration and Taylor series approximations

*[CEGM1000 MUDE](http://mude.citg.tudelft.nl/)*

*Written by: Anna Störiko*

*Due: `<day of week>`, `<month>` `<day>`, `<year>`.*

## Part 1


Explain the salt dilution method

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

time = np.array([  0.,  10.,  20.,  30.,  41.,  50.,  60.,  70.,  78.,  90., 100.,
       110., 120., 130., 140., 150., 170., 180., 190., 200.])
concentration = np.array([0, 0.02, 0.1, 0.4, 0.7, 0.6, 0.4, 0.3, 0.22, 0.18, 0.15, 0.12, 0.1, 0.08, 0.06, 0.04, 0.01, 0, 0, 0])
dt = np.diff(time)
plt.plot(time, concentration, marker="o")

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

### Task 1.1

Compute the area under the concentration curve by numerical integration. Compare two numerical integration methods of your choice.

</p>
</div>

In [ ]:
# Compute the area of the concentration curve with the trapezoidal rule
trapezoid_areas = (concentration[:-1] + concentration[1:]) / 2 * dt
total_area_trapz = np.sum(trapezoid_areas)

# Compute the area of the concentration curve with the left Riemann sum
total_area_left_riemann = np.sum(concentration[:-1] * dt)

# Compute the area of the concentration curve with the right Riemann sum
total_area_right_riemann = np.sum(concentration[1:] * dt)

print(
f"""
Total area under the concentration curve:
Left Riemann sum: {total_area_left_riemann:.2f}
Right Riemann sum: {total_area_right_riemann:.2f}
Trapezoidal rule: {total_area_trapz:.2f}
"""
)

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

### Task 1.2

Which integration method is best suited for this dataset? Why?

</p>
</div>

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

### Task 1.3

Compute the discharge based on the concentration data.

</p>
</div>

In [ ]:
mass = 3
discharge = mass / total_area_trapz
print(discharge)

To model the concentration curve, we fitted an analytical solution of the advection-dispersion-equation to the concentration data.
As a consistency if the model is a good approximation to the data, we want to compare the area under the simulated concentration curve to the area under the curve obtained with measurements.

In [ ]:
# You do not need to change anything in this cell

def advection_dispersion(t, x, v, D, m, A):
    "Compute the solution to the advection-dispersion equation"
    solution = (
        m / (A * v)
        * x / (np.sqrt(4 * np.pi * D * t**3))
        * np.exp(-((x - v * t) ** 2) / (4 * D * t))
    )
    return np.where(t > 0, solution, 0)

# Fit the model to the data to obtain the parameters v, D and A
x = 20
f = lambda time, v, D, A: advection_dispersion(time, x, v, D, mass, A)
popt, pcov = curve_fit(
    f, xdata=time, ydata=concentration, p0=(0.2, 0.1, 1), bounds=(0, np.inf)
)
v, D, A = popt

# Plot the fitted curve along with the data
time_grid = np.linspace(0, time[-1], 100)
c_analytical = advection_dispersion(time_grid, x, v, D, mass, A)
plt.plot(time_grid, c_analytical, label="simulation")
plt.scatter(time, concentration, label="measurements")
plt.xlabel("time [s]")
plt.ylabel("concentration [mg/L]")
plt.legend()



<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 1.**
    
Evaluate the integral of the simulated concentration curve from time 0 to x seconds, using Simpson's rule.
Observe how the result changes with different numbers of integration intervals.
</p>
</div>

In [ ]:
# Integrate the concentration curve with Simpson's rule
intervals = np.arange(2, 50, step=2)
integrals = []
for n_intervals in intervals:
    t_end = time[-1]
    dt = t_end / n_intervals
    t = np.linspace(0, t_end, n_intervals+1)
    f_evaluated = advection_dispersion(t, x, v, D, mass, A)
    terms = [
        (f_evaluated[2 * i - 2] + 4 * f_evaluated[2 * i - 1] + f_evaluated[2 * i]) / 6 * 2 * dt
        for i in range(1, n_intervals // 2 + 1)
    ]
    integral = sum(terms)
    integrals.append(integral)

In [ ]:
plt.plot(intervals, integrals, marker="o")
plt.xlabel("Nr. of integration intervals")
plt.ylabel("area under the curve");

## Part 2: Taylor series approximation

Taylor series are a powerful tool to approximate functions.
Throughout this week’s chapter in the text book, you have seen how Taylor series can be used to derive numerical approximations for derivatives and estimate the errors of numerical integration and differentiation techniques.

In this assignment, we will have a closer look at how this approximation works.
The visualizations you creat will hopefully help you to build some intuition for the meaning of a Taylor series.

We will approximate the natural logarithm by a number of Taylor polynomials of increasing order.

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.1**

This is a pen-and-paper exercise. Write down the terms first four terms of the Taylor series around $x_0=1$ for

$$f(x) = \ln(x)$$
</p>
</div>

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>


**Task 2.2**

Before continuing with Taylor series approximation, plot the expression $f(x)=\ln(x)$ in the interval $[0, 3]$. This will be used as benchmark to assess your approximations. We will want to produce plots that will include each successive term of the Taylor approximation to see how the approximation improves as we include more terms in the Taylor series.

</p>
</div>

In [ ]:
x = np.linspace(0, 3, 200)

def f(x):
    return np.log(x)

plt.plot(x, f(x))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title("Plot of $f(x) = \\ln(x)$");

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.3**

Implement a Python function that evaluats the nth derivative of $\ln(x)$, so we can use it to compute the Talor polynomial.
Complete the function template in the code cell below. 

</p>
</div>

In [ ]:
import math

def fn(x, n):
    """Compute the nth derivative of ln(x) at point x"""
    return (-1)**(n+1) * math.factorial(n-1) * x**(-n)

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.4**

Define the expansion point $x_0$ in Python and provide Python expressions for the Taylor polynomials of first, second, third and fourth order.

</p>
</div>

In [ ]:
x0 = # YOUR_CODE_HERE
taylor_1 = # YOUR_CODE_HERE
taylor_2 = # YOUR_CODE_HERE
taylor_3 = # YOUR_CODE_HERE
taylor_4 = # YOUR_CODE_HERE

In [ ]:
x0 = 1
taylor_1 = f(x0) + fn(x0, 1) * (x - x0)
taylor_2 = taylor_1 + fn(x0, 2) * (x - x0) ** 2 / math.factorial(2)
taylor_3 = taylor_2 + fn(x0, 3) * (x - x0) ** 3 / math.factorial(3)
taylor_4 = taylor_3 + fn(x0, 4) * (x - x0) ** 4 / math.factorial(4)

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.5**

Fill in the code below to plot the function $f(x)$ along with the Taylor polynomials of increasing order.

</p>
</div>

In [ ]:
plt.plot(x, f(x), label="$f(x) = \\ln(x)$", color="k")
plt.plot(x, taylor_1, label="First Order", color="C0")
plt.plot(x, taylor_2, label="Second Order", color="C1")
plt.plot(x, taylor_3, label="Third Order", color="C2")
plt.plot(x, taylor_4, label="Fourth Order", color="C3")


plt.scatter(
    [x0], [f(x0)], color="k", marker="o", label=f"Expansion Point ($x = {x0:0.2f}$)"
)

plt.xlabel("x")
plt.ylabel("$f(x)$")
plt.title("Taylor polynomials of $f(x) = \\ln(x)$")
plt.legend()

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.6**

Look at the plot you just created. Which Taylor polynomial approximates the function $f(x)$ the best? How does depend on the distance from the expansion point $x_0$?

</p>
</div>

To further analyze the error introduced by truncating the Taylor series, we can evaluate the absolute difference between the function $f(x)$ and the Taylor polynomials:

$$\text{error} =|f(x)-T_n|\,,$$

where $T_n$ refers to the Taylor polynomial of $n$th order.

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 2.7**

Use your Taylor series approximations and the analytic expression for $f(x)$ to determine the absolute error. Plot the error against $x$ and vary the $x$- and $y$-limits. Are the larger orders Taylor polynomials always more accurate? 

</p>
</div>

In [ ]:
error_1 = np.abs(f(x) - taylor_1)
error_2 = np.abs(f(x) - taylor_2)
error_3 = np.abs(f(x) - taylor_3)
error_4 = np.abs(f(x) - taylor_4)

plt.plot(x, error_1, label="First Order", color="C0")
plt.plot(x, error_2, label="Second Order", color="C1")
plt.plot(x, error_3, label="Third Order", color="C2")
plt.plot(x, error_4, label="Fourth Order", color="C3")

plt.xlabel("x")
plt.ylabel("Absolute Error: $f(x)-\\mathrm{Taylor~polynomial}$")
plt.title("Absolute Error of Taylor Series Approximations")
plt.legend()

## Part 3: Deriving numerical derivatives from Taylor series expansions

After we had a closer look at the meaning of Taylor series, you will now practice how you can apply Taylor series to derive expressions for numerical derivatives.

<div style="background-color:#AABAB2; color: black; width:90%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>

**Task 3.1**
    
Derive the backward difference method such that it is *second order* accurate. Refer to the book for an illustration of this approach with first order accuracy. Insert an image of your math below.

*You don't have to further use the result in this assignment, but it is useful to understand the approach and preparing for the exam.*

Tips:
- You will have to combine Taylor series expansions for the function at two different points.
- At what order do you need to truncate the Taylor series to get a second order accurate method?    
</p>
</div>

<div style="background-color:#FAE99E; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 90%">
<p>

**Solution 3.1**
    
For backward euler we need to evaluate at $x_{i−1}$ and $x_{i−2}$ around $x_i$. This yields the following Taylor approximation:
    
$$f(x_{i-1})\approx f(x_{i})+(x_{i-1}-x_i)\frac{\partial f(x_{i})}{\partial x} +\frac{(x_{i-1}-x_i)^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(x_{i-1}-x_i)^3$$
$$f(x_{i-2})\approx f(x_{i})+(x_{i-2}-x_i)\frac{\partial f(x_{i})}{\partial x} +\frac{(x_{i-2}-x_i)^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(x_{i-2}-x_i)^3$$

    
We set $\Delta x = x_i - x_{i-1}$ for all $i$, which also means: $2\Delta x = x_i-x_{i-2}$
    
$$f(x_{i-1})\approx f(x_{i})-\Delta x\frac{\partial f(x_{i})}{\partial x} +\frac{\Delta x^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(\Delta x)^3$$
$$f(x_{i-2})\approx f(x_{i})-2\Delta x\frac{\partial f(x_{i})}{\partial x} +\frac{4\Delta x^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(\Delta x)^3$$
    
To get rid of the term including the second derivative, we multiply the first expression by 4 and subtract the second expression:
    
$$
\begin{aligned}
4f(x_{i-1})-f(x_{i-2})&\approx (4-1)f(x_{i})-(4-2)\Delta x\frac{\partial f(x_{i})}{\partial x} + (4-4)\frac{\Delta x^2}{2}\frac{\partial^2 f(x_i)}{\partial x^2}+\mathcal{O}(\Delta x)^3\\
&= 3f(x_{i})-2\Delta x\frac{\partial f(x_{i})}{\partial x} + \mathcal{O}(\Delta x)^3
\end{aligned}
$$

Bring the derivative to the left side and all terms involving $f(x)$ to the right side:
$$ 2\Delta x\frac{\partial f(x_{i})}{\partial x} \approx 3f(x_i)-4f(x_{i-1})+f(x_{i-2}) +\mathcal{O}(\Delta x)^3$$
$$\frac{\partial f(x_{i})}{\partial x} \approx \frac{3f(x_i)-4f(x_{i-1})+f(x_{i-2})}{2\Delta x} +\mathcal{O}(\Delta x)^2$$

</p></div>

In [ ]:
# Provided code which is not to be edited by students
import numpy as np

In [ ]:
# The code given to students can be indicated with:
# with a cell tag `assignment`.
# These cells will be removed from the solution notebook.
# The assignment notebook will be generated after a push to main and
# will be stored on the branch `assignment`.
# Places where student should write their code are marked with `### YOUR CODE HERE ###` for filling in a single line
# or with `### YOUR CODE LINES HERE` for filling in multiple lines

import numpy as np

a = [### YOUR CODE HERE ###]
    
### YOUR CODE LINES HERE

print('First 20 fibonacci numbers:', ### YOUR CODE HERE ###)

<div style="background-color:#FAE99E; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 90%">

$\text{Solution x.x:}$

Having trouble understanding how the Gauss-Newton scheme works? Do you want to visualize how the parameters and model change on each iteration? Try uncommenting the cell below to create an interactive plot!

It is set up to visualize the model on each iteration. You can gain insight into the way convergence occurs by changing the value of <code>initial_guess_alternative</code> and rerunning the cell.
</p></div>

Cells containing a solution block (with color ` #FAE99E`) will be removed in the assignment notebook.

In [ ]:
# Solution code (in a separate cell) is indicated with
# a cell tag 'solution'.
# These cells will be removed from the assignment notebook.
# The assignment notebook will be generated after a push to main and
# will be stored on the branch `solution`.

a = [0,2]

for i in range(2, 20):
    a.append(a[i-1] + a[i-2])

print('First 20 fibonacci numbers:', a)

<div style="background-color:#FAE99E; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px; width: 90%">
<p>
End of solution.
</p>
</div>

Cells containing a solution block (with color ` #FAE99E`) will be removed in the assignment notebook.

<div style="margin-top: 50px; padding-top: 20px; border-top: 1px solid #ccc;">
  <div style="display: flex; justify-content: flex-end; gap: 20px; align-items: center;">
    <a rel="MUDE" href="http://mude.citg.tudelft.nl/">
      <img alt="MUDE" style="width:100px; height:auto;" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png" />
    </a>
    <a rel="TU Delft" href="https://www.tudelft.nl/en/ceg">
      <img alt="TU Delft" style="width:100px; height:auto;" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png" />
    </a>
    <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">
      <img alt="Creative Commons License" style="width:88px; height:auto;" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
    </a>
  </div>
  <div style="font-size: 75%; margin-top: 10px; text-align: right;">
    &copy; Copyright 2025 <a rel="MUDE" href="http://mude.citg.tudelft.nl/">MUDE</a> TU Delft. 
    This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">CC BY 4.0 License</a>.
  </div>
</div>